In [ ]:
#Osman Ebrahim
#EBR23592118
#18/02/25

In [2]:
import time # Used to introduce delays, allowing step-by-step visualization of robot movement 
import random # Used to generate random obstacles, no-entry zones, and one-way streets 
import heapq # Provides priority queue functionality, used for the A* algorithm
class SmartDeliveryRobot:
    def __init__(self, size, delivery_points, start_position, obstacles, no_entry, one_way):
        self.size = size # Grid size (NxN)
        self.delivery_points = set(delivery_points) # Set of coordinates to deliver parcels
        self.position = start_position # Current robot position
        self.delivered = set() # Tracks delivered points
        self.obstacles = obstacles # Positions with obstacles
        self.no_entry = no_entry # Restricted zones
        self.one_way = one_way # One-way direction constraints
        self.actions = [] # Records robot's actions for later review

        # Robot movement functions (left, right, up, down)
    def move_left(self):
        x, y = self.position
        if x > 1 and (x - 1, y) not in self.obstacles and (x - 1, y) not in self.no_entry and self.one_way.get((x, y)) != "➡️":
            self.position = (x - 1, y)
            self.actions.append(f"Moved left to {self.position}")

    def move_right(self):
        x, y = self.position
        if x < self.size and (x + 1, y) not in self.obstacles and (x + 1, y) not in self.no_entry and self.one_way.get((x, y)) != "⬅️":
            self.position = (x + 1, y)
            self.actions.append(f"Moved right to {self.position}")

    def move_up(self):
        x, y = self.position
        if y < self.size and (x, y + 1) not in self.obstacles and (x, y + 1) not in self.no_entry and self.one_way.get((x, y)) != "⬇️":
            self.position = (x, y + 1)
            self.actions.append(f"Moved up to {self.position}")

    def move_down(self):
        x, y = self.position
        if y > 1 and (x, y - 1) not in self.obstacles and (x, y - 1) not in self.no_entry and self.one_way.get((x, y)) != "⬆️":
            self.position = (x, y - 1)
            self.actions.append(f"Moved down to {self.position}")

    def deliver(self):
        if self.position in self.delivery_points and self.position not in self.delivered:
            self.delivered.add(self.position)
            self.actions.append(f"Delivered at {self.position} ✅")

    def all_delivered(self):
        return self.delivered == self.delivery_points

    def find_shortest_path(self, start, goal):
        """Use A* algorithm to find the shortest path from start to goal."""
        def heuristic(a, b):
            return abs(a[0] - b[0]) + abs(a[1] - b[1])  # Manhattan distance as heuristic

        def is_valid_move(x, y):
            if x < 1 or x > self.size or y < 1 or y > self.size:
                return False
            if (x, y) in self.obstacles or (x, y) in self.no_entry:
                return False
            if self.one_way.get((x, y)) == "⬅️" and (x - 1, y) != start:
                return False
            if self.one_way.get((x, y)) == "⬆️" and (x, y - 1) != start:
                return False
            return True

        open_list = [] # Priority queue of nodes to explore
        heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start))  # (priority, g_cost, position)
        came_from = {} # Tracks the path
        g_costs = {start: 0} # Cost from start to current node
        f_costs = {start: heuristic(start, goal)}

        while open_list:
            _, g, current = heapq.heappop(open_list) #Select node with lowest f_cost

            if current == goal: # Path found
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.reverse()
                return path

            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]: # Explore adjacent tiles
                next_position = (current[0] + dx, current[1] + dy)
                if is_valid_move(*next_position):
                    tentative_g = g + 1
                    if next_position not in g_costs or tentative_g < g_costs[next_position]:
                        g_costs[next_position] = tentative_g
                        f_cost = tentative_g + heuristic(next_position, goal)
                        f_costs[next_position] = f_cost
                        came_from[next_position] = current
                        heapq.heappush(open_list, (f_cost, tentative_g, next_position))

        return []  # No path found

# Utility function to ensure valid numeric input from the user
def get_valid_input(prompt, min_val, max_val):
    while True:
        try:
            value = int(input(prompt))
            if min_val <= value <= max_val:
                return value
            print(f"Please enter a number between {min_val} and {max_val}.")
        except ValueError:
            print("Invalid input. Enter a valid number.")
            
# Generates user-defined delivery points
def generate_delivery_points(size):
    num_deliveries = get_valid_input(f"Enter number of delivery points (1-{size**2}): ", 1, size**2)
    delivery_points = set()
    
    while len(delivery_points) < num_deliveries:
        x = get_valid_input(f"Enter x-coordinate for delivery {len(delivery_points) + 1}: ", 1, size)
        y = get_valid_input(f"Enter y-coordinate for delivery {len(delivery_points) + 1}: ", 1, size)
        if (x, y) in delivery_points:
            print("Delivery point already exists. Choose a different coordinate.")
        else:
            delivery_points.add((x, y))
    
    return delivery_points

# Randomly generates obstacles on the grid, excluding delivery points
def generate_obstacles(size, delivery_points):
    num_obstacles = random.randint(1, size)  
    obstacles = set()

    while len(obstacles) < num_obstacles:
        x, y = random.randint(1, size), random.randint(1, size)
        if (x, y) not in delivery_points:
            obstacles.add((x, y))

    return obstacles

# Generates restricted no-entry zones
def generate_no_entry_zones(size, delivery_points, obstacles):
    num_no_entry = random.randint(0, size // 2)  
    no_entry = set()

    while len(no_entry) < num_no_entry:
        x, y = random.randint(1, size), random.randint(1, size)
        if (x, y) not in delivery_points and (x, y) not in obstacles:
            no_entry.add((x, y))

    return no_entry

# Generates random one-way streets with direction restrictions
def generate_one_way_streets(size, delivery_points, obstacles, no_entry):
    one_way = {}
    directions = ["➡️", "⬅️", "⬆️", "⬇️"]

    for _ in range(random.randint(0, size)):  
        x, y = random.randint(1, size), random.randint(1, size)
        if (x, y) not in delivery_points and (x, y) not in obstacles and (x, y) not in no_entry:
            one_way[(x, y)] = random.choice(directions)

    return one_way

# Displays the current grid with all environmental elements
def display_grid(size, delivery_points, robot_position, delivered_points, obstacles, no_entry, one_way):
    print("\nCurrent Environment:") #this is the enviroment
    for y in range(size, 0, -1):  # Rows from top to bottom
        row = []
        for x in range(1, size + 1):
            if (x, y) == robot_position:
                row.append("🤖")  # Robot position
            elif (x, y) in delivered_points:
                row.append("Delivered ✅")  # Delivered parcels
            elif (x, y) in delivery_points:
                row.append("Delivery 📦")  # Pending delivery
            elif (x, y) in obstacles:
                row.append("🚧")  # Obstacles
            elif (x, y) in no_entry:
                row.append("⛔")  # No-entry zones
            elif (x, y) in one_way:
                row.append(one_way[(x, y)])  # One-way directions
            else:
                row.append("Clear")  # Empty space
        print("  ".join(row))
    print("\n")

def toggle_features(obstacles, no_entry, one_way):
    """Allow toggling of obstacles, no-entry zones, and one-way streets for testing."""
    feature = input("Which feature would you like to toggle? (obstacles/no-entry/one-way): ").lower()
    
    if feature == "obstacles":
        action = input("Toggle obstacles (on/off): ").lower()
        if action == "on":
            print("Obstacles are now visible on the map.")
            obstacles = generate_obstacles(size, delivery_points)
        elif action == "off":
            print("Obstacles are now hidden.")
            obstacles = set()
    
    elif feature == "no-entry":
        action = input("Toggle no-entry zones (on/off): ").lower()
        if action == "on":
            print("No-entry zones are now visible on the map.")
            no_entry = generate_no_entry_zones(size, delivery_points, obstacles)
        elif action == "off":
            print("No-entry zones are now hidden.")
            no_entry = set()
    
    elif feature == "one-way":
        action = input("Toggle one-way streets (on/off): ").lower()
        if action == "on":
            print("One-way streets are now visible on the map.")
            one_way = generate_one_way_streets(size, delivery_points, obstacles, no_entry)
        elif action == "off":
            print("One-way streets are now hidden.")
            one_way = set()

    return obstacles, no_entry, one_way

# Handles autonomous delivery navigation
def autonomous_navigation(robot):
    while not robot.all_delivered():
        remaining_deliveries = robot.delivery_points - robot.delivered
        target = min(remaining_deliveries, key=lambda p: abs(p[0] - robot.position[0]) + abs(p[1] - robot.position[1]))

        # Check if delivery point is accessible
        if not robot.find_shortest_path(robot.position, target):
            print(f"❌ Delivery point {target} is blocked! Trying alternative...")
            continue  # Skip to next delivery if the point is blocked

        # Get the shortest path to the target delivery point using A* algorithm
        path = robot.find_shortest_path(robot.position, target)

        if path:
            for next_position in path:
                while robot.position != next_position:
                    x, y = robot.position
                    tx, ty = next_position

                    if x < tx:
                        robot.move_right()
                    elif x > tx:
                        robot.move_left()
                    elif y < ty:
                        robot.move_up()
                    elif y > ty:
                        robot.move_down()

                    display_grid(robot.size, robot.delivery_points, robot.position, robot.delivered, robot.obstacles, robot.no_entry, robot.one_way)
                    time.sleep(0.5) # Slow down visualization

            robot.deliver() # Deliver at the target
            print(f"📦 Delivered at {robot.position} ✅\n")
            display_grid(robot.size, robot.delivery_points, robot.position, robot.delivered, robot.obstacles, robot.no_entry, robot.one_way)
            time.sleep(0.5)

    print("\n🎉 All deliveries completed successfully! 🎉")
    print(f"Total actions taken: {len(robot.actions)}")
    for action in robot.actions:
        print(action)

# Entry point of the program
def main():
    size = get_valid_input("Enter grid size (1-6): ", 1, 6) # User-defined grid size
    delivery_points = generate_delivery_points(size) # Create delivery points

    obstacles = generate_obstacles(size, delivery_points) # Generate obstacles
    no_entry = generate_no_entry_zones(size, delivery_points, obstacles) # Generate no-entry zones
    one_way = generate_one_way_streets(size, delivery_points, obstacles, no_entry)  # Create one-way streets

    start_x = get_valid_input("Enter robot's starting x-coordinate: ", 1, size)
    start_y = get_valid_input("Enter robot's starting y-coordinate: ", 1, size)

    robot = SmartDeliveryRobot(size, delivery_points, (start_x, start_y), obstacles, no_entry, one_way)
    display_grid(size, delivery_points, robot.position, robot.delivered, obstacles, no_entry, one_way)

    print("\n🚀 Starting autonomous delivery process...\n")
    autonomous_navigation(robot) # Start the delivery process
    
# Run the program
if __name__ == "__main__":
    main()


Enter grid size (1-6): 6
Enter number of delivery points (1-36): 2
Enter x-coordinate for delivery 1: 1
Enter y-coordinate for delivery 1: 1
Enter x-coordinate for delivery 2: 1
Enter y-coordinate for delivery 2: 1
Delivery point already exists. Choose a different coordinate.
Enter x-coordinate for delivery 2: 1
Enter y-coordinate for delivery 2: 6
Enter robot's starting x-coordinate: 3
Enter robot's starting y-coordinate: 4

Current Environment:
Delivery 📦  Clear  Clear  Clear  Clear  Clear
⛔  🚧  Clear  Clear  Clear  Clear
Clear  Clear  🤖  Clear  🚧  Clear
Clear  Clear  Clear  🚧  Clear  Clear
Clear  Clear  Clear  Clear  Clear  Clear
Delivery 📦  Clear  Clear  Clear  Clear  Clear



🚀 Starting autonomous delivery process...


Current Environment:
Delivery 📦  Clear  Clear  Clear  Clear  Clear
⛔  🚧  🤖  Clear  Clear  Clear
Clear  Clear  Clear  Clear  🚧  Clear
Clear  Clear  Clear  🚧  Clear  Clear
Clear  Clear  Clear  Clear  Clear  Clear
Delivery 📦  Clear  Clear  Clear  Clear  Clear



Curren